### The Kalman Filter operates in a loop of two main steps:
### Prediction: Estimate the next state using the current estimate (prior knowledge).
### Update: Refine the prediction using the newly observed measurement, optimally balancing model prediction and new data.
### This loop continues for each incoming measurement, resulting in continuous improvement of the state estimate.


In [1]:
print("test")

test


In [4]:
import cv2
import numpy as np
from ultralytics import YOLO


# ============================
# Kalman Filter2D
# ============================

class Kalman2D:
    def __init__(self, dt=1.0):
        self.x = np.zeros((4, 1))  # state [x, y, vx, vy]
        self.P = np.eye(4) * 1000  # error covariance
        self.F = np.array([
            [1, 0, dt, 0],
            [0, 1, dt, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])  # state transition
        self.H = np.array([
            [1, 0, 0, 0],
            [0, 1, 0, 0]
        ])  # measurement matrix
        self.Q = np.eye(4) * 0.1  # process noise
        self.R = np.eye(2) * 10   # measurement noise
        self.I = np.eye(4)        # identity

    def predict(self):
        self.x = self.F @ self.x
        self.P = self.F @ self.P @ self.F.T + self.Q
        return self.x

    def update(self, z):
        z = np.reshape(z, (2, 1))
        y = z - (self.H @ self.x)
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)
        self.x = self.x + K @ y
        self.P = (self.I - K @ self.H) @ self.P
        return self.x


# ============================
# YOLO + Kalman Tracking
# ============================

model = YOLO("yolov8n.pt")
cap = cv2.VideoCapture(0)
kf = Kalman2D()
pred_w, pred_h = 60, 60  # initial predicted box size

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, stream=True)
    detected = False

    for r in results:
        for box in r.boxes:
            # YOLO box coordinates
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            cx = (x1 + x2) / 2
            cy = (y1 + y2) / 2

            detected = True

            # Kalman predict + update
            kf.predict()
            est = kf.update([cx, cy])
            ex, ey = int(est[0][0]), int(est[1][0])

            # Draw YOLO box (real detection)
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (100, 200, 0), 2)
            cv2.putText(frame, "YOLO", (int(x1), int(y1)-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (100, 200, 0), 2)

            # Draw Kalman predicted center
            cv2.circle(frame, (ex, ey), 5, (0, 255, 0), -1)
            cv2.putText(frame, "KF", (ex - 10, ey - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Update predicted box size from YOLO box
            pred_w = int(x2 - x1)
            pred_h = int(y2 - y1)

            # Draw predicted box (Kalman)
            cv2.rectangle(frame,
                          (ex - pred_w // 2, ey - pred_h // 2),
                          (ex + pred_w // 2, ey + pred_h // 2),
                          (255, 100, 0), 2)  # predicted box in different color

    # If no detection, just predict
    if not detected:
        kf.predict()
        ex, ey = int(kf.x[0][0]), int(kf.x[1][0])
        cv2.circle(frame, (ex, ey), 5, (0, 0, 255), -1)
        cv2.putText(frame, "Predict", (ex - 30, ey - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Draw predicted box
        cv2.rectangle(frame,
                      (ex - pred_w // 2, ey - pred_h // 2),
                      (ex + pred_w // 2, ey + pred_h // 2),
                      (255, 100, 0), 2)

    cv2.imshow("YOLO + Kalman Tracking", frame)
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 1 person, 15.4ms
Speed: 3.7ms preprocess, 15.4ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 30.8ms
Speed: 4.4ms preprocess, 30.8ms inference, 3.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 13.6ms
Speed: 4.8ms preprocess, 13.6ms inference, 5.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 23.4ms
Speed: 2.3ms preprocess, 23.4ms inference, 4.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 13.6ms
Speed: 3.0ms preprocess, 13.6ms inference, 4.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 25.5ms
Speed: 3.8ms preprocess, 25.5ms inference, 5.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 26.1ms
Speed: 5.2ms preprocess, 26.1ms inference, 3.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 17.3ms
Speed: 2.3ms preprocess, 17.3ms inference, 7.8ms postprocess per image at shape (1, 3, 48

In [1]:
import cv2
import numpy as np
from ultralytics import YOLO

# --------------------------------------------------------
# 1. إعدادات Kalman Filter
# --------------------------------------------------------
class Kalman2D:
    def __init__(self, dt=1/30): 
        self.x = np.zeros((4, 1))  
        self.F = np.array([
            [1, 0, dt, 0],
            [0, 1, 0, dt],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])
        self.H = np.array([
            [1, 0, 0, 0],
            [0, 1, 0, 0]
        ])
        self.P = np.eye(4) * 1000 
        self.Q = np.eye(4) * 0.1
        self.R = np.eye(2) * 10

    def predict(self):
        self.x = self.F @ self.x
        self.P = self.F @ self.P @ self.F.T + self.Q
        return self.x

    def update(self, z):
        z = np.reshape(z, (2, 1))
        y = z - (self.H @ self.x)
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)
        self.x = self.x + K @ y
        self.I = np.eye(4)
        self.P = (self.I - K @ self.H) @ self.P
        return self.x

# --------------------------------------------------------
# 2. تشغيل التتبع والحفظ
# --------------------------------------------------------

model_path = r"C:\Users\mmwmn\Downloads\best.pt" 
model = YOLO(model_path)

video_source = r"C:\Users\mmwmn\Downloads\ملخص مباراة مانشستر يونايتد ووست هام (1-1) _ الدوري الإنجليزي - الجولة 14(720P_HD).mp4"
cap = cv2.VideoCapture(video_source)

# --- إعدادات الحفظ (الجديد هنا) ---
# 1. الحصول على خصائص الفيديو الأصلي (العرض، الطول، عدد الفريمات)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# 2. تحديد اسم ومكان الحفظ
output_path = r"C:\Users\mmwmn\Downloads\output_tracking.mp4"

# 3. إعداد أداة الكتابة (Codec)
# mp4v مناسب لملفات mp4
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
# ----------------------------------

kf = Kalman2D(dt=1/fps) # تعديل الزمن ليكون دقيقاً حسب الفيديو
last_w, last_h = 50, 50 

print("بدأ العمل... اضغط ESC للإيقاف")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 1. التنبؤ
    pred = kf.predict()
    pred_x, pred_y = int(pred[0]), int(pred[1])

    # 2. تشغيل YOLO
    results = model(frame, verbose=False)
    
    best_box = None
    max_conf = 0.0

    for res in results:
        for box in res.boxes:
            conf = float(box.conf[0])
            # تأكد أن كلاس الكرة هو 0 في الموديل الخاص بك، لو مختلف غير الرقم
            cls = int(box.cls[0]) 
            if conf > 0.3 and cls == 0: 
                if conf > max_conf:
                    max_conf = conf
                    best_box = box

    # 3. التحديث والرسم
    if best_box is not None:
        x1, y1, x2, y2 = map(int, best_box.xyxy[0])
        cx = (x1 + x2) // 2
        cy = (y1 + y2) // 2
        
        last_w = x2 - x1
        last_h = y2 - y1

        kf.update([cx, cy])

        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(frame, f"YOLO {max_conf:.2f}", (x1, y1-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    
    # رسم الكالمان
    cv2.circle(frame, (pred_x, pred_y), 5, (0, 255, 0), -1)
    cv2.rectangle(frame, 
                  (pred_x - last_w//2, pred_y - last_h//2),
                  (pred_x + last_w//2, pred_y + last_h//2),
                  (0, 255, 0), 1)
    cv2.putText(frame, "Kalman", (pred_x, pred_y-10), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)

    # --- حفظ الفريم في الفيديو الجديد ---
    out.write(frame)
    # ------------------------------------

    cv2.imshow("Ball Tracking", frame)
    
    if cv2.waitKey(1) & 0xFF == 27:
        break

# تنظيف وإغلاق
cap.release()
out.release() # ضروري جداً لإغلاق ملف الفيديو وحفظه
cv2.destroyAllWindows()
print(f"تم الانتهاء! الفيديو محفوظ في: {output_path}")

بدأ العمل... اضغط ESC للإيقاف


C:\Users\mmwmn\AppData\Local\Temp\ipykernel_14260\1612280400.py:77: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_x, pred_y = int(pred[0]), int(pred[1])


تم الانتهاء! الفيديو محفوظ في: C:\Users\mmwmn\Downloads\output_tracking.mp4


In [3]:
import cv2
import numpy as np
from ultralytics import YOLO

# --------------------------------------------------------
# 1. إعدادات Kalman Filter المحسّنة
# --------------------------------------------------------
class Kalman2D:
    def __init__(self, dt=1/30):
        self.x = np.zeros((4, 1))  # [x, y, vx, vy]

        self.F = np.array([
            [1, 0, dt, 0],
            [0, 1, 0, dt],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])

        self.H = np.array([
            [1, 0, 0, 0],
            [0, 1, 0, 0]
        ])

        self.P = np.eye(4) * 500
        self.Q = np.eye(4) * 0.01   # ضوضاء الحركة (أقل = توقع أكثر نعومة)
        self.R = np.eye(2) * 5      # ضوضاء القياس (YOLO)

    def predict(self):
        self.x = self.F @ self.x
        self.P = self.F @ self.P @ self.F.T + self.Q
        return self.x

    def update(self, z):
        z = np.reshape(z, (2, 1))
        y = z - (self.H @ self.x)
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)
        self.x = self.x + K @ y
        self.P = (np.eye(4) - K @ self.H) @ self.P
        return self.x

# --------------------------------------------------------
# 2. إعداد YOLO + الفيديو + الحفظ
# --------------------------------------------------------

model_path = r"C:\Users\mmwmn\Downloads\best.pt"
model = YOLO(model_path)

video_source = r"C:\Users\mmwmn\Downloads\ملخص مباراة مانشستر يونايتد ووست هام (1-1) _ الدوري الإنجليزي - الجولة 14(720P_HD).mp4"
cap = cv2.VideoCapture(video_source)

fps = cap.get(cv2.CAP_PROP_FPS)
frame_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

output_path = r"C:\Users\mmwmn\Downloads\output_tracking.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_w, frame_h))

kf = Kalman2D(dt=1/fps)
trajectory = []      # لتسجيل المسار
frame_id = 0         # Frame skipping
no_detect_frames = 0 # فقدان الكشف
last_w, last_h = 50, 50

print("بدأ التتبع...")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1

    # -----------------------------
    # 1) توقع كالمان
    # -----------------------------
    pred = kf.predict()
    pred_x, pred_y = int(pred[0]), int(pred[1])
    pred_point = np.array([pred_x, pred_y])

    # -----------------------------
    # 2) تشغيل YOLO كل فريمين (تحسين الأداء)
    # -----------------------------
    if frame_id % 2 == 0:
        results = model(frame, verbose=False)
    else:
        results = []

    # -----------------------------
    # 3) أفضل صندوق = ثقة - مسافة من كالمان
    # -----------------------------
    best_box = None
    best_score = -9999

    for r in results:
        for box in r.boxes:

            cls = int(box.cls[0])
            if cls != 0:  # الكلاس 0 = الكرة
                continue

            conf = float(box.conf[0])
            if conf < 0.15:
                continue

            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2

            dist = np.linalg.norm(np.array([cx, cy]) - pred_point)
            score = conf - (dist * 0.0008)

            if score > best_score:
                best_score = score
                best_box = (x1, y1, x2, y2, cx, cy, conf)

    # -----------------------------
    # 4) تحديث كالمان أو الاستمرار بالتوقع فقط
    # -----------------------------
    if best_box is not None:
        x1, y1, x2, y2, cx, cy, conf = best_box

        kf.update([cx, cy])
        last_w = x2 - x1
        last_h = y2 - y1
        no_detect_frames = 0

        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(frame, f"YOLO {conf:.2f}", (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    else:
        no_detect_frames += 1

    # -----------------------------
    # 5) إعادة تهيئة عند فقدان التتبع
    # -----------------------------
    if no_detect_frames > 20:
        kf.x = np.zeros((4, 1))

    # -----------------------------
    # 6) الرسم: كالمان + المسار
    # -----------------------------
    cv2.circle(frame, (pred_x, pred_y), 5, (0, 255, 0), -1)
    cv2.rectangle(frame,
                  (pred_x - last_w//2, pred_y - last_h//2),
                  (pred_x + last_w//2, pred_y + last_h//2),
                  (0, 255, 0), 2)
    cv2.putText(frame, "Kalman", (pred_x+10, pred_y-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    trajectory.append((pred_x, pred_y))
    if len(trajectory) > 50:
        trajectory.pop(0)

    for i in range(1, len(trajectory)):
        cv2.line(frame, trajectory[i-1], trajectory[i], (0,255,255), 2)

    # الحفظ + العرض
    out.write(frame)
    cv2.imshow("Ball Tracking", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("✔️ تم الانتهاء! الملف محفوظ في:", output_path)


بدأ التتبع...


C:\Users\mmwmn\AppData\Local\Temp\ipykernel_14260\680111830.py:79: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_x, pred_y = int(pred[0]), int(pred[1])


✔️ تم الانتهاء! الملف محفوظ في: C:\Users\mmwmn\Downloads\output_tracking.mp4


In [4]:
import cv2
import numpy as np
from ultralytics import YOLO

# =========================================================
# 1️⃣ Kalman Filter (نسخة محسّنة)
# =========================================================
class Kalman2D:
    def __init__(self, dt=1/30):
        self.x = np.zeros((4, 1))
        self.F = np.array([
            [1, 0, dt, 0],
            [0, 1, 0, dt],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])
        self.H = np.array([
            [1, 0, 0, 0],
            [0, 1, 0, 0]
        ])
        self.P = np.eye(4) * 500      # ثبات أكبر
        self.Q = np.eye(4) * 0.01     # ضوضاء حركة أقل
        self.R = np.eye(2) * 5        # دمج أفضل مع القياس
        self.I = np.eye(4)

    def predict(self):
        self.x = self.F @ self.x
        self.P = self.F @ self.P @ self.F.T + self.Q
        return self.x

    def update(self, z):
        z = np.reshape(z, (2, 1))
        y = z - (self.H @ self.x)
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)

        self.x = self.x + K @ y
        self.P = (self.I - K @ self.H) @ self.P
        return self.x


# =========================================================
# 2️⃣ تحميل YOLO
# =========================================================
model_path = r"C:\Users\mmwmn\Downloads\best.pt" 
model = YOLO(model_path)

# =========================================================
# 3️⃣ قراءة الفيديو + التحضير للحفظ
# =========================================================
video_source = r"C:\Users\mmwmn\Downloads\ملخص مباراة مانشستر يونايتد ووست هام (1-1) _ الدوري الإنجليزي - الجولة 14(720P_HD).mp4"
cap = cv2.VideoCapture(video_source)

frame_width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

output_path = r"C:\Users\mmwmn\Downloads\output_tracking.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

kf = Kalman2D(dt=1/fps)

print("🎥 بدأ التتبع... اضغط ESC للإيقاف")

# =========================================================
# 4️⃣ تحسينات إضافية
# =========================================================
trajectory = []     # خط مسار الكرة
max_traj_len = 40    # عدد النقاط المحفوظة

frame_id = 0
no_detect_frames = 0
last_w, last_h = 50, 50

# =========================================================
# 5️⃣ الحلقة الأساسية
# =========================================================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1

    # 1. توقع Kalman
    pred = kf.predict()
    pred_x, pred_y = int(pred[0]), int(pred[1])
    pred_point = np.array([pred_x, pred_y])

    # 2. YOLO (كل فريمين)
    if frame_id % 2 == 0:
        results = model(frame, verbose=False)
    else:
        results = []

    best_box = None
    best_score = -9999

    # 3. اختيار المربع الأفضل (ثقة + قرب)
    for res in results:
        for box in res.boxes:
            conf = float(box.conf[0])
            cls = int(box.cls[0])

            if cls != 0:  
                continue
            
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2
            
            dist = np.linalg.norm(np.array([cx, cy]) - pred_point)
            score = conf - (dist * 0.001)

            if score > best_score:
                best_score = score
                best_box = (x1, y1, x2, y2, cx, cy, conf)

    # 4. تحديث Kalman أو زيادة عداد الفقدان
    if best_box is not None:
        x1, y1, x2, y2, cx, cy, conf = best_box
        kf.update([cx, cy])

        last_w = x2 - x1
        last_h = y2 - y1
        no_detect_frames = 0

        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 80, 0), 2)
        cv2.putText(frame, f"YOLO {conf:.2f}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 80, 0), 2)

    else:
        no_detect_frames += 1

    # 5. إعادة تهيئة Kalman عند فقدان التتبع
    if no_detect_frames > 20:
        kf.x = np.zeros((4,1))
        last_w, last_h = 50, 50

    # 6. رسم الكالمان
    cv2.circle(frame, (pred_x, pred_y), 5, (0, 255, 0), -1)
    cv2.rectangle(frame,
                  (pred_x - last_w // 2, pred_y - last_h // 2),
                  (pred_x + last_w // 2, pred_y + last_h // 2),
                  (0, 255, 0), 1)
    cv2.putText(frame, "Kalman", (pred_x + 10, pred_y - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # 7. رسم المسار (trajectory)
    trajectory.append((pred_x, pred_y))
    if len(trajectory) > max_traj_len:
        trajectory.pop(0)

    for i in range(1, len(trajectory)):
        cv2.line(frame, trajectory[i-1], trajectory[i], (0, 255, 255), 2)

    # 8. حفظ الفريم
    out.write(frame)

    cv2.imshow("Ball Tracking", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"✔️ تم الانتهاء! الفيديو محفوظ في:\n{output_path}")


🎥 بدأ التتبع... اضغط ESC للإيقاف


C:\Users\mmwmn\AppData\Local\Temp\ipykernel_14260\2351906065.py:88: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_x, pred_y = int(pred[0]), int(pred[1])


✔️ تم الانتهاء! الفيديو محفوظ في:
C:\Users\mmwmn\Downloads\output_tracking.mp4
